### Varios

### Convertir los datasets de las competencias a json y separarlos 75/25 train/test
- Se crea el modelo evomsa pre-entrenado haha2021_Es.evomsa
- En Single f1_score_macro  = 0.823913749710607
- En Muliple f1_score_macro = 
  - haha2018, mexa3t, misoginia + exist21_model
- En Muliple f1_score_macro = 
  - mexa3t, misoginia + exist21_model
- La mejora es apenas 0

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)

In [3]:
df = pd.read_csv("../FakeDeS2021_train.csv", usecols=['Text', 'Category']) #,   header=0, nrows=50)# , nrows=30, header=1)
df.rename(columns = {'Category':'klass', 'Text':'text'}, inplace = True)
df["klass"] = np.where(df["klass"] == 'True', 0, 1)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['klass'], train_size=0.75)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((507,), (169,), (507,), (169,))

In [2]:
df_train = pd.read_json('../mexa3t2018_aggress_Es_train.json', lines=True)
df_train = df_train[["text", "klass"]]
df_test = pd.read_json('../mexa3t2018_aggress_Es_test.json', lines=True)
df_test = df_test[["text", "klass"]]

In [3]:
np.unique(df_train['klass'])
# 1 es humor
# 0 No es humor

array([0, 1], dtype=int64)

In [50]:
# 
#df_train["klass"] = np.where(df_train["klass"] == 'N', 1, 0)
#df_test["klass"] = np.where(df_test["klass"] !=0, 1, 0)
#np.unique(df_train['klass'])

array([0, 1])

In [4]:
print(df_train.shape, df_test.shape)

(5389, 2) (2311, 2)


In [5]:
df_train

,text,klass
0,"Que putas se supone que haga si Sam S y coldplay solo irán a cdmx ya que gasté mas de 10,000 pesos en puros boletos de este año y del 18'",0
1,"#NoEsPosibleQue yo vuleva a caer dos veces , naaa ni q tuvieras tanta suerte!",0
2,Se pasaron de verga la cague muy feo en el gold xdd izi win era,0
3,me recomendó mucho este libro de fotos sobre los 10 años de la marcha marica en Chile y hoy por fin lo co…,0
4,HIJOS DE LA CHINGADA PARA SUS PUTOS FESTIVALES RUIDOSOS E ILEGALES SI TIENEN MADRE NO?A LA CARCEL ZULEYMA Y MONRATA,1
...,...,...
5384,"Virtudes de los políticos;Idiotas,Rateros,Sin Sesos,Rateros,Cinicos y no festejan el 10 mayo por no tener Madre chulada de espécimen. <URL>",1
5385,@USUARIO @USUARIO El video es tomado en la calle principal: Santa Madre De Dios y abarca el tramo señalado,0
5386,"Los maestros siempre vienen conmigo a hablar mal de otros maestros. Los niños, a decirme quién les gusta. Mi única respuesta: ME VALE VERGA.",1
5387,Tiran a dar como un puñal tus palabras en mi ombligo. Tú estas loca y yo estoy loco de atar por mis celos retroactivos. 🎶🎶🎶 @luisramiromusic,0


In [6]:
from EvoMSA.base import EvoMSA
from microtc.utils import save_model
from sklearn import metrics
from microtc.utils import load_model
from microtc.utils import save_model

In [7]:
evo = EvoMSA(TR=True, B4MSA=False, lang='es', Emo=True, Aggress=True, models=None)

X_train, y_train = df_train['text'], df_train['klass']
X_test, y_test = df_test['text'], df_test['klass']

# Pq a .squeeze() ? 

In [8]:
evo.fit(X_train.squeeze(), y_train.squeeze())
pred = evo.predict(X_test.squeeze())
print(metrics.f1_score(y_test, pred, average="macro"))

100%|██████████| 30/30 [00:06<00:00,  4.41it/s]

0.7806387046501216


In [9]:
# crear modelo pre-entrenado en Sigle par evitar efecto cascada
save_model(evo, '..\\..\\single_models\\mexa3t2018_aggress_single_Es.evomsa') 

### Multiple...
---

In [36]:
stacked_method = "sklearn.svm.LinearSVC"
# usar = "sklearn.naive_bayes.BernoulliNB"
# usar = "sklearn.naive_bayes.MultinomialNB"

exist21_model = [load_model('../../single_models/exist21_single_Es.evomsa'), stacked_method]
haha21_model = [load_model('../../single_models/haha21_single_Es.evomsa'), stacked_method]
davincis22_model = [load_model('../../single_models/davincis22_single_Es.evomsa'), stacked_method]
meoffendes21_model = [load_model('../../single_models/meoffendes21_single_Es.evomsa'), stacked_method]

mm = [exist21_model, haha21_model, davincis22_model, meoffendes21_model]

evo = EvoMSA(TR=True, B4MSA=False, lang='es', Emo=True, Aggress=True, stacked_method=stacked_method, models= mm)

In [37]:
evo.fit(X_train.squeeze(), y_train.squeeze())
pred = evo.predict(X_test.squeeze())

100%|██████████| 3/3 [00:04<00:00,  1.48s/it]

100%|██████████| 3/3 [00:04<00:00,  1.49s/it]

100%|██████████| 3/3 [00:03<00:00,  1.33s/it]

100%|██████████| 3/3 [00:04<00:00,  1.41s/it]

100%|██████████| 3/3 [00:01<00:00,  1.53it/s]

100%|██████████| 3/3 [00:02<00:00,  1.43it/s]

100%|██████████| 3/3 [00:01<00:00,  1.56it/s]

100%|██████████| 3/3 [00:02<00:00,  1.48it/s]

100%|██████████| 7/7 [00:00<00:00, 1754.00it/s]


In [21]:
print(metrics.f1_score(y_test, pred, average="macro"))

0.7945544328523051


In [21]:
print(metrics.f1_score(y_test, pred, average="macro"))

0.8969856602137946


In [86]:
# crear modelo pre-entrenado
save_model(evo, '..\\..\\models\\haha2021_Es.evomsa') 

In [ ]:
# probarlo
_ = load_model('..\\..\\models\\haha2021_Es.evomsa') 
pred = _.predict(X_test.squeeze())
print(metrics.f1_score(y_test, pred, average="macro"))